# Creamos un DataFrame CON PANDAS a partir de fichero de texto de tipo .txt #

## Importación de librerías que se precisan ##

In [18]:
import pandas as pd

## EDA : Preparación y limpieza de los datos ##

### Valores iniciales de variables que se utilizarán para: ###
- Identificar los archivos de información en crudo (.txt)
- Componer el nombre del fichero resultante de todo el proceso (.csv)

In [19]:
# Cumplimentar Parámetros según estación a procesar
estacion = 'AvFrancia'
idEstacion = str(46250047)
desdeAño = 2009
hastaAño = 2022
#Nombre del directorio de datos
PATH_read = "datos"
PATH_write = "datoscsv" 

### Consideraciones respecto a las rutas de los archivos ###

file path in Windows : `rootdirectory\\subdirectory\\subsubdirectory\\filename.extension`

file path in Mac and Linux : `rootdirectory/subdirectory/subsubdirectory/filename.extension`

En Windows, hay dos barras invertidas porque cada barra invertida debe escaparse con otra barra invertida.

Esto se puede gestionar mediante el método os.path.join().

Este método maneja los separadores según el sistema operativo.

In [20]:
import os
pathname = os.path.join(PATH_read)
print(pathname)


datos


### Primera fase : ###
Eliminación de líneas no relevantes y prescindibles del fichero crudo (.txt)

In [21]:
# Eliminar lineas innecesarias
for año in range(desdeAño, hastaAño + 1):
    fichero = pathname+'/MDEST' + str(idEstacion) + str(año) + '.txt'
    lineas = []
    # Codificación entornos WINDOWS
    # with open(fichero, encoding='cp1252') as ftxt:
    # Codificación entornos LINUX
    with open(fichero, encoding='cp1252') as ftxt: 
        lineas = ftxt.readlines()
        del lineas[4]
        del lineas[2]
        del lineas[1]
        del lineas[0]
    # escribir archivo de salida una vez hechos los reemplazos
    with open(fichero, "w") as ftxt:
        ftxt.writelines(lineas)

### Segunda fase : ###
- Reemplazo de caracteres para estandarizazar los valores arreglo a los tipos de datos.
- Corrección de contenido en el campo fecha de algunos de los ficheros.

In [22]:
# Reemplazar comas por puntos en ficheros de entrada
for año in range(desdeAño, hastaAño + 1):
    fichero = pathname + '/MDEST' + str(idEstacion) + str(año) + '.txt'
    salida = []
    with open(fichero) as ftxt:
        lineas = ftxt.readlines()
        for linea in lineas:
            linea = linea.replace(',', '.')
            linea = linea.replace(' 0:00', '')
            salida.append(linea)
    # escribir archivo de salida una vez hechos los reemplazos
    with open(fichero, "w") as ftxt:
        ftxt.writelines(salida)

In [23]:
# Crear dataframe vacio
df = pd.DataFrame()
# Crear una lista de columnas vacía
columnas = []

In [24]:
# Cargar todos los archivos txt de la estación
for año in range(desdeAño, hastaAño + 1):
    fichero = pathname + '/MDEST' + str(idEstacion) + str(año) + '.txt'
    dfaux = pd.read_table(fichero, decimal=".")
    # Comprobar si hay nuevas columnas
    print(f'>>>>>>>>> Año:{año} ', end =',') 
    for c in dfaux.columns:
        if c not in columnas:
            print(f' {c}', end= ',')
            columnas.append(c)
    print()
    df = pd.concat([df, dfaux])

>>>>>>>>> Año:2009 , FECHA, SO2, CO, PM1, O3, Veloc., NO2, NO, PM2.5, PM10, NOx, Direc.,
>>>>>>>>> Año:2010 ,
>>>>>>>>> Año:2011 , Unnamed: 2, Unnamed: 4, Unnamed: 6, Unnamed: 8, Unnamed: 10, Unnamed: 12, HCT, Unnamed: 14, CH4, Unnamed: 16, HNM, Unnamed: 18, Unnamed: 20, Unnamed: 22, Veloc.máx., Unnamed: 24,
>>>>>>>>> Año:2012 ,
>>>>>>>>> Año:2013 ,
>>>>>>>>> Año:2014 ,
>>>>>>>>> Año:2015 ,
>>>>>>>>> Año:2016 ,
>>>>>>>>> Año:2017 ,
>>>>>>>>> Año:2018 ,
>>>>>>>>> Año:2019 ,
>>>>>>>>> Año:2020 ,
>>>>>>>>> Año:2021 ,
>>>>>>>>> Año:2022 ,


In [25]:
# Convertir a numerica todas las columnas excepto la primera (FECHA)
columnas = df.columns.to_list()
del columnas[0]
for c in columnas:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Convertir la fecha a formato datetime
df['FECHA'] = pd.to_datetime(df['FECHA'], format="%d/%m/%Y")

df.insert(0, 'idEstacion', idEstacion)
df.insert(1, 'estacion', estacion)

df

,idEstacion,estacion,FECHA,SO2,CO,PM1,O3,Veloc.,NO2,NO,...,HCT,Unnamed: 14,CH4,Unnamed: 16,HNM,Unnamed: 18,Unnamed: 20,Unnamed: 22,Veloc.máx.,Unnamed: 24
0,46250047,AvFrancia,2009-01-01,3.0,0.4,11.0,18.0,1.1,59.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,46250047,AvFrancia,2009-01-02,3.0,0.5,13.0,13.0,1.1,74.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,46250047,AvFrancia,2009-01-03,3.0,0.4,12.0,10.0,0.7,70.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46250047,AvFrancia,2009-01-04,3.0,0.4,6.0,28.0,1.7,56.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46250047,AvFrancia,2009-01-05,3.0,0.4,7.0,16.0,1.0,75.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,46250047,AvFrancia,2022-03-27,4.0,0.1,NaN,91.0,2.0,5.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,46250047,AvFrancia,2022-03-28,4.0,0.1,NaN,70.0,NaN,13.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,46250047,AvFrancia,2022-03-29,4.0,0.1,NaN,64.0,2.6,13.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,46250047,AvFrancia,2022-03-30,4.0,0.1,NaN,59.0,NaN,13.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4792 entries, 0 to 89
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   idEstacion   4792 non-null   object        
 1   estacion     4792 non-null   object        
 2   FECHA        4792 non-null   datetime64[ns]
 3   SO2          4430 non-null   float64       
 4   CO           4400 non-null   float64       
 5   PM1          1899 non-null   float64       
 6   O3           4676 non-null   float64       
 7   Veloc.       4768 non-null   float64       
 8   NO2          4354 non-null   float64       
 9   NO           4354 non-null   float64       
 10  PM2.5        3199 non-null   float64       
 11  PM10         3199 non-null   float64       
 12  NOx          4354 non-null   float64       
 13  Direc.       4789 non-null   float64       
 14  Unnamed: 2   0 non-null      float64       
 15  Unnamed: 4   0 non-null      float64       
 16  Unnamed:

In [27]:
df.describe()

,SO2,CO,PM1,O3,Veloc.,NO2,NO,PM2.5,PM10,NOx,...,HCT,Unnamed: 14,CH4,Unnamed: 16,HNM,Unnamed: 18,Unnamed: 20,Unnamed: 22,Veloc.máx.,Unnamed: 24
count,4430.000000,4400.000000,1899.000000,4676.000000,4768.000000,4354.000000,4354.000000,3199.000000,3199.000000,4354.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351.000000,0.0
mean,3.444470,0.179818,6.713007,49.605646,1.392638,26.545935,9.282729,9.979681,16.227884,40.409968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.710541,NaN
std,1.308899,0.091423,4.510530,17.850890,0.842296,16.184791,12.427016,6.406778,10.380034,32.989418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.955221,NaN
min,0.000000,0.000000,0.000000,5.000000,0.200000,1.000000,0.000000,0.000000,1.000000,2.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,NaN
25%,3.000000,0.100000,4.000000,38.000000,0.900000,14.000000,2.000000,6.000000,10.000000,19.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.450000,NaN
50%,3.000000,0.200000,6.000000,51.000000,1.300000,23.000000,5.000000,9.000000,14.000000,30.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.500000,NaN
75%,4.000000,0.200000,8.500000,62.000000,1.700000,35.000000,10.000000,13.000000,20.000000,50.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.600000,NaN
max,14.000000,0.700000,45.000000,115.000000,10.900000,110.000000,143.000000,49.000000,209.000000,296.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.600000,NaN


In [28]:
df.isnull().sum()/len(df)*100

idEstacion       0.000000
estacion         0.000000
FECHA            0.000000
SO2              7.554257
CO               8.180301
PM1             60.371452
O3               2.420701
Veloc.           0.500835
NO2              9.140234
NO               9.140234
PM2.5           33.242905
PM10            33.242905
NOx              9.140234
Direc.           0.062604
Unnamed: 2     100.000000
Unnamed: 4     100.000000
Unnamed: 6     100.000000
Unnamed: 8     100.000000
Unnamed: 10    100.000000
Unnamed: 12    100.000000
HCT            100.000000
Unnamed: 14    100.000000
CH4            100.000000
Unnamed: 16    100.000000
HNM            100.000000
Unnamed: 18    100.000000
Unnamed: 20    100.000000
Unnamed: 22    100.000000
Veloc.máx.      92.675292
Unnamed: 24    100.000000
dtype: float64

A partir de aqui grabamos el .cvs en el directorio datoscsv

In [29]:
pathname2 = os.path.join(PATH_write)
print(pathname2)

datoscsv


In [30]:
df.to_csv(pathname2 + '/' + idEstacion + estacion + '.csv', sep=';', index=False)

Cargamos el archivo .csv en el dataframe df2

In [31]:
df2 = pd.read_csv(pathname2 + '/' + idEstacion + estacion + '.csv', sep=';', decimal='.')
df2

,idEstacion,estacion,FECHA,SO2,CO,PM1,O3,Veloc.,NO2,NO,...,HCT,Unnamed: 14,CH4,Unnamed: 16,HNM,Unnamed: 18,Unnamed: 20,Unnamed: 22,Veloc.máx.,Unnamed: 24
0,46250047,AvFrancia,2009-01-01,3.0,0.4,11.0,18.0,1.1,59.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,46250047,AvFrancia,2009-01-02,3.0,0.5,13.0,13.0,1.1,74.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,46250047,AvFrancia,2009-01-03,3.0,0.4,12.0,10.0,0.7,70.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46250047,AvFrancia,2009-01-04,3.0,0.4,6.0,28.0,1.7,56.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46250047,AvFrancia,2009-01-05,3.0,0.4,7.0,16.0,1.0,75.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4787,46250047,AvFrancia,2022-03-27,4.0,0.1,NaN,91.0,2.0,5.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4788,46250047,AvFrancia,2022-03-28,4.0,0.1,NaN,70.0,NaN,13.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4789,46250047,AvFrancia,2022-03-29,4.0,0.1,NaN,64.0,2.6,13.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4790,46250047,AvFrancia,2022-03-30,4.0,0.1,NaN,59.0,NaN,13.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4792 entries, 0 to 4791
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   idEstacion   4792 non-null   int64  
 1   estacion     4792 non-null   object 
 2   FECHA        4792 non-null   object 
 3   SO2          4430 non-null   float64
 4   CO           4400 non-null   float64
 5   PM1          1899 non-null   float64
 6   O3           4676 non-null   float64
 7   Veloc.       4768 non-null   float64
 8   NO2          4354 non-null   float64
 9   NO           4354 non-null   float64
 10  PM2.5        3199 non-null   float64
 11  PM10         3199 non-null   float64
 12  NOx          4354 non-null   float64
 13  Direc.       4789 non-null   float64
 14  Unnamed: 2   0 non-null      float64
 15  Unnamed: 4   0 non-null      float64
 16  Unnamed: 6   0 non-null      float64
 17  Unnamed: 8   0 non-null      float64
 18  Unnamed: 10  0 non-null      float64
 19  Unname

In [33]:
df2.describe()

,idEstacion,SO2,CO,PM1,O3,Veloc.,NO2,NO,PM2.5,PM10,...,HCT,Unnamed: 14,CH4,Unnamed: 16,HNM,Unnamed: 18,Unnamed: 20,Unnamed: 22,Veloc.máx.,Unnamed: 24
count,4792.0,4430.000000,4400.000000,1899.000000,4676.000000,4768.000000,4354.000000,4354.000000,3199.000000,3199.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351.000000,0.0
mean,46250047.0,3.444470,0.179818,6.713007,49.605646,1.392638,26.545935,9.282729,9.979681,16.227884,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.710541,NaN
std,0.0,1.308899,0.091423,4.510530,17.850890,0.842296,16.184791,12.427016,6.406778,10.380034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.955221,NaN
min,46250047.0,0.000000,0.000000,0.000000,5.000000,0.200000,1.000000,0.000000,0.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,NaN
25%,46250047.0,3.000000,0.100000,4.000000,38.000000,0.900000,14.000000,2.000000,6.000000,10.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.450000,NaN
50%,46250047.0,3.000000,0.200000,6.000000,51.000000,1.300000,23.000000,5.000000,9.000000,14.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.500000,NaN
75%,46250047.0,4.000000,0.200000,8.500000,62.000000,1.700000,35.000000,10.000000,13.000000,20.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.600000,NaN
max,46250047.0,14.000000,0.700000,45.000000,115.000000,10.900000,110.000000,143.000000,49.000000,209.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.600000,NaN


In [34]:
df2.isnull().sum()/len(df)*100

idEstacion       0.000000
estacion         0.000000
FECHA            0.000000
SO2              7.554257
CO               8.180301
PM1             60.371452
O3               2.420701
Veloc.           0.500835
NO2              9.140234
NO               9.140234
PM2.5           33.242905
PM10            33.242905
NOx              9.140234
Direc.           0.062604
Unnamed: 2     100.000000
Unnamed: 4     100.000000
Unnamed: 6     100.000000
Unnamed: 8     100.000000
Unnamed: 10    100.000000
Unnamed: 12    100.000000
HCT            100.000000
Unnamed: 14    100.000000
CH4            100.000000
Unnamed: 16    100.000000
HNM            100.000000
Unnamed: 18    100.000000
Unnamed: 20    100.000000
Unnamed: 22    100.000000
Veloc.máx.      92.675292
Unnamed: 24    100.000000
dtype: float64